<a href="https://colab.research.google.com/github/AUT-Student/CN-Project/blob/main/CN_Project_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [2]:
import pandas as pd
import numpy as np

In [3]:
import torch

In [4]:
torch.__version__

'1.13.1+cu116'

In [5]:
!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch-geometric
!pip install torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-1.13.0+cu116.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 58.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 KB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 25.1 MB/s eta 0:00:00
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773302 sha256=59de481dacd7d4f70e069936426342ffd7e122bf7c3a02f9106d5afb00a74c8d
  Stored in directory: /root/.cache/pip/wheels/59/a3/20/198928106d3169865ae73afcbd3d3d1796cf6b429b55c65378
Successfully built 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 873.9/873.9 KB 18.1 MB/s eta 0:00:00


In [6]:
import torch_geometric

# Dataset

## Create

In [7]:
from torch_geometric.datasets.dblp import DBLP

In [8]:
dblp = DBLP("/content")

Extracting /content/raw/DBLP_processed.zip
Processing...
Done!


## Overview

In [9]:
dblp.data

HeteroData(
  author={
    x=[4057, 334],
    y=[4057],
    train_mask=[4057],
    val_mask=[4057],
    test_mask=[4057]
  },
  paper={ x=[14328, 4231] },
  term={ x=[7723, 50] },
  conference={ num_nodes=20 },
  (author, to, paper)={ edge_index=[2, 19645] },
  (paper, to, author)={ edge_index=[2, 19645] },
  (paper, to, term)={ edge_index=[2, 85810] },
  (paper, to, conference)={ edge_index=[2, 14328] },
  (term, to, paper)={ edge_index=[2, 85810] },
  (conference, to, paper)={ edge_index=[2, 14328] }
)

In [10]:
dblp.num_edge_features

{('author', 'to', 'paper'): 0,
 ('paper', 'to', 'author'): 0,
 ('paper', 'to', 'term'): 0,
 ('paper', 'to', 'conference'): 0,
 ('term', 'to', 'paper'): 0,
 ('conference', 'to', 'paper'): 0}

In [12]:
dblp.num_features

{'author': 334, 'paper': 4231, 'term': 50, 'conference': 0}

In [13]:
dblp.data['author']

{'x': tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'y': tensor([2, 2, 3,  ..., 0, 0, 0]), 'train_mask': tensor([False, False, False,  ..., False, False, False]), 'val_mask': tensor([False, False,  True,  ..., False, False, False]), 'test_mask': tensor([ True,  True, False,  ...,  True,  True,  True])}

In [28]:
dblp.data['paper']

{'x': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}

In [28]:
set(dblp.data['author']["y"].numpy())

{0, 1, 2, 3}

In [149]:
(sum(dblp.data['author']["train_mask"].numpy()), sum(dblp.data['author']["test_mask"].numpy()), sum(dblp.data['author']["val_mask"].numpy()))

(400, 3257, 400)

In [29]:
dblp.num_features

{'author': 334, 'paper': 4231, 'term': 50, 'conference': 0, 'y': 0}

In [35]:
dblp.data['author'].num_features

334

In [109]:
author_x = dblp.data["author"].x

In [110]:
pta_edges = dblp[0].edge_index_dict[('paper','to','author')]
paper_auther_data = pd.DataFrame(pta_edges.numpy().T, columns=["paper", "author"])

In [111]:
author_edges = []
for authors in paper_auther_data.groupby("paper")["author"].apply(np.array):
  for author1 in authors:
    for author2 in authors:
      if author1==author2:continue
      author_edges.append([author1, author2])

author_edges = torch.Tensor(np.array(author_edges).T).to(torch.long)

# GCN

In [144]:
import torch
from torch import Tensor
from torch_geometric.nn import GCNConv
from torch.nn import Linear, ReLU

dataset = dblp

class GCN(torch.nn.Module):
    def __init__(self, dimentions, number_gcn, number_fc):
        super().__init__()
        assert(len(dimentions) == number_gcn+number_fc+1)
        self.number_gcn = number_gcn
        self.number_fc = number_fc
        self.layers_gcn = []
        self.layers_fc = []

        for in_dim, out_dim in zip(dimentions[:number_gcn],
                                   dimentions[1:number_gcn+1]):
          
          self.layers_gcn.append(GCNConv(in_dim, out_dim))
          
        self.layers_gcn = torch.nn.ModuleList(self.layers_gcn)
        
        for in_dim, out_dim in zip(dimentions[number_gcn:],
                                   dimentions[number_gcn+1:]):
          
          self.layers_fc.append(Linear(in_dim, out_dim))
        
        self.layers_fc = torch.nn.ModuleList(self.layers_fc)
        

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        for i in range(self.number_gcn):
          x = self.layers_gcn[i](x, edge_index).relu()

        for i in range(self.number_fc):
          x = self.layers_fc[i](x).sigmoid()

        return x

model = GCN([334, 64, 32, 16, 4], 2, 2) #GCN(dataset["author"].num_features, 16, dataset.num_classes)

In [145]:
import torch.nn.functional as F

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(200):
    pred = model(author_x, author_edges)
    loss = F.cross_entropy(pred[dblp.data["author"].train_mask], dblp.data["author"].y[dblp.data["author"].train_mask])

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [146]:
with torch.no_grad():
  pred = model(author_x, author_edges).argmax(dim=-1)

In [147]:
accs = []
for mask in [dblp.data["author"].train_mask, dblp.data["author"].val_mask, dblp.data["author"].test_mask]:
    accs.append(int((pred[mask] == dblp.data["author"].y[mask]).sum()) / int(mask.sum()))

In [148]:
accs

[0.99, 0.715, 0.7448572305802886]